In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import animation
import glob
import re

In [ ]:
# folder names
hundred_runs = "data - run - 100"
bots_runs = "data and images - run 1"
personality_run = "data-bot-personality-50"
fake_threshold_run = "fake"
fake_threshold_effect = "fake effect and threshold"
moderation = "moderation 3-7"
move = "move"
move2 = 'move-no-exp'


print(df.columns)
f_100 = glob.glob(hundred_runs+"/*")
f_bots = glob.glob(bots_runs+"/*")
f_personality = glob.glob(personality_run+"/*")
f_fake_th = glob.glob(fake_threshold_run+"/*")
f_fake_th_eff = glob.glob(fake_threshold_effect+"/*")
f_mod = glob.glob(moderation+"/*")

In [ ]:
def get_by_iteration(folder_name):
    file_list = glob.glob(folder_name+"/*")
    ret_list = {}
    for filename in file_list:
        original_name = filename
        filename = filename.replace(folder_name, '')
        filename = filename[1:-4]
        splits = filename.split('-')[1:]
        iteration = re.sub('\D', '', splits[0])
        factors = []
        for split in splits[1:]:
            factor = re.sub('\D', '', split)
            factors.append(factor)
        factor_name = '+'.join(factors)
        if factor_name in ret_list.keys():
            ret_list[factor_name].append(original_name)
        else:
            ret_list[factor_name] = [original_name]
    return ret_list  

In [ ]:
def get_bot_percentage(data_item):
    pattern = r'\]\s+\['
    splits = re.split(pattern, data_item)
#     print(l)
    data = []
    for split in splits:
        string_data = split.replace('[', "").replace(']', "")
        item = np.fromstring(string_data, dtype=float, sep=' ')
        data.append(item)
    return np.asarray(data)

# print(get_bot_percentage('test 1]    [test2'))

In [ ]:
filenames = get_by_iteration(hundred_runs)
avgs = []
# column_name = "bot_percentage"
column_name = "Agent_emotion_dist"
for key in filenames.keys():
    group_data = []
    group = filenames[key]
    for file in group[:1]:
        df = pd.read_csv(file)
        new_col = []
        for data in df[column_name]:
            print(data)
            new_col.append(get_bot_percentage(data))
        group_data.append(new_col)

    bot_avg = np.mean( np.array(group_data), axis=0 )
    avgs.append(bot_avg)

In [ ]:
def plot_bots(arr, time_step, title, current_min=0, current_max=1, filename=None):
    fig, ax = plt.subplots()
    im = ax.imshow(arr[time_step], interpolation="nearest", cmap='bwr', vmin=current_min, vmax=current_max)
    ax.set_yticks([-0.5, 0, 0.5, 1, 1.5])
    ax.set_yticklabels(['', 'AGAINST', '', 'PRO',''])
    ax.set_xticks(list(range(6)))
    ax.set_xticklabels(['LOW UNPOP,\nMOST UNREL','LOW UNPOP,\nSOME UNREL','MED UNPOP,\nMOST UNREL',
                        'MED UNPOP,\nSOME UNREL','HIGH UNPOP,\nMOST UNREL','HIGH UNPOP,\nSOME UNREL'], size=6)
    ax.set_title(f'{title}\nStep: {time_step}')
    plt.colorbar(im)
    if filename != None:
        fig.savefig(filename)


In [ ]:
from operator import sub

filenames = get_by_iteration(move2)
avgs = []
# column_name = "bot_percentage"
column_name = "Post_belief_dist"
col2 = "Agent_belief_dist"

# print(data_list)
plt_title = "Belief Difference - more movement"
for key in filenames.keys():
    group_data = []
    group = filenames[key]
    for file in group:
        df = pd.read_csv(file)
        dl1 = df[column_name]
        dl2 = df[col2]
        new_col = []
        new_col2 = []
        for data in dl1:
            new_col.append(get_bot_percentage(data))
        for data in dl2:
            new_col2.append(get_bot_percentage(data))
        group_data.append(np.absolute((np.array(new_col)-np.array(new_col2))))

    bot_avg = np.mean( np.array(group_data), axis=0 )
    avgs.append(bot_avg)

for i in range(0, len(avgs[1]), 1):
    print(i)
    plot_bots(avgs[1], i, plt_title, 0, 4, filename="images/slides/bdiff-0-"+str(i)+'.png')

In [ ]:
filenames = get_by_iteration(fake_threshold_run)
avgs = []
column_name = "Post_emotion_dist"
# column_name = "bot_percentage"
# column_name = "fake_threshold_effect"

data_list = df[column_name]
plt_title = "Post Sentiment"
for key in filenames.keys():
    group_data = []
    group = filenames[key]
    for file in group:
        df = pd.read_csv(file)
        new_col = []
        for data in data_list:
            new_col.append(get_bot_percentage(data))
        group_data.append(new_col)

    bot_avg = np.mean( np.array(group_data), axis=0 )
    avgs.append(bot_avg)

for i in range(0, len(avgs[1]), 2):
    print(i)
    plot_bots(avgs[1], i, plt_title, 1, 5, filename="images/slides/p-"+str(i)+'.png')


In [ ]:
# For animation
import imageio
import os

images = []

def get_nums(filename):
    return int(re.sub('\D', '', filename))

filenames = glob.glob("images/slides/*")
filenames.sort(key = lambda x:get_nums(x))
print(filenames)

for filename in filenames:
    images.append(imageio.imread(filename))
imageio.mimsave('post_happy.gif', images, duration=0.25, loop=1)